# import python packages

In [83]:
import pandas as pd
import datetime as dt
import shutil
import os

# AEP frequencies

In [84]:
AEP_frequency = pd.DataFrame({'frequency':['frequent', 'frequent', 'frequent', 'frequent', 'frequent', 'intermediate', 'intermediate', 'rare', 'rare'], 
                              'EY':[1, 0.69, 0.5, 0.22, 0.2, 0.11, 0.05, 0.02, 0.01],
                              'AEP_in_percentage':[63.21, 50, 39.35, 20, 18.13, 10, 5, 2, 1], 
                              'AEP_1_in_X_years':[1.58, 2, 2.54, 5, 5.52, 10, 20, 50, 100],
                              'ARI':[1, 1.44, 2, 4.48, 5, 9.49, 19.5, 49.5, 99.5]})
AEP_frequency

,frequency,EY,AEP_in_percentage,AEP_1_in_X_years,ARI
0,frequent,1.00,63.21,1.58,1.00
1,frequent,0.69,50.00,2.00,1.44
2,frequent,0.50,39.35,2.54,2.00
3,frequent,0.22,20.00,5.00,4.48
4,frequent,0.20,18.13,5.52,5.00
5,intermediate,0.11,10.00,10.00,9.49
6,intermediate,0.05,5.00,20.00,19.50
7,rare,0.02,2.00,50.00,49.50
8,rare,0.01,1.00,100.00,99.50


# read inputs

## precipitation burst duration

In [85]:
df_statistics = pd.read_csv('../Inputs_FE_Assignment_3/ECsouth/ECsouth_AllStats.csv')
df_statistics = df_statistics.dropna(axis=0)
column_statistics = [df_statistics.columns.to_list()[index_1].replace(' ', '_') for index_1 in range(len(df_statistics.columns.to_list()))]
column_statistics = [column_statistics[index_1].replace('(', '') for index_1 in range(len(column_statistics))]
column_statistics = [column_statistics[index_1].replace(')', '') for index_1 in range(len(column_statistics))]
column_statistics = [column_statistics[index_1].replace('.', '') for index_1 in range(len(column_statistics))]
column_statistics = [column_statistics[index_1].replace('%', 'percentage') for index_1 in range(len(column_statistics))]
df_statistics.rename(columns={df_statistics.columns.to_list()[index_1]: column_statistics[index_1] for index_1 in range(len(column_statistics))}, inplace=True)
df_statistics

,Event_ID,Region,Region_source,Burst_Duration_min,Burst_Loading,Original_Burst_Depth_mm,AEP_Window,AEP_source_percentage,Burst_Start_Date,Burst_End_Date,DB_Event_Reference_No,DB_Pluviograph_Reference_No,Offical_Gauge,Lat,Long
0,4380.0,East Coast (South),East Coast (South),10.0,1.0,15.50,frequent,14.4679,3/25/2009 17:45,3/25/2009 17:50,113650.0,1828.0,qcd_563064_233,-32.2375,150.6306
1,4382.0,East Coast (South),East Coast (South),10.0,1.0,18.80,frequent,14.4924,12/28/1999 17:05,12/28/1999 17:10,39777.0,517.0,qcd_061250,-32.6296,151.5919
2,4384.0,East Coast (South),East Coast (South),10.0,1.0,16.80,frequent,14.5144,2/4/2005 9:30,2/4/2005 9:35,42925.0,556.0,qcd_068102,-34.4869,150.4019
3,4385.0,East Coast (South),East Coast (South),10.0,2.0,18.50,frequent,15.6473,3/27/1999 22:30,3/27/1999 22:35,114063.0,1835.0,qcd_563079_231,-33.7944,150.5083
4,4386.0,East Coast (South),East Coast (South),10.0,2.0,11.00,frequent,47.6798,4/28/1993 14:05,4/28/1993 14:10,113366.0,1824.0,qcd_563056_231,-34.0333,150.2153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2856.0,East Coast (South),East Coast (North),10080.0,1.0,595.23,rare,1.0865,5/1/1996 0:05,5/8/1996 0:00,26738.0,351.0,qcd_040223,-27.4178,153.1142
716,5064.0,East Coast (South),East Coast (North),10080.0,2.0,589.88,rare,1.2899,1/6/1968 17:10,1/13/1968 17:05,24711.0,329.0,qcd_040093,-26.1831,152.6414
717,5069.0,East Coast (South),East Coast (North),10080.0,2.0,491.26,rare,2.7020,1/6/1968 4:10,1/13/1968 4:05,23354.0,313.0,qcd_039128,-24.9069,152.3230
718,5071.0,East Coast (South),East Coast (North),10080.0,2.0,400.84,rare,2.9538,1/7/1968 11:45,1/14/1968 11:40,27412.0,358.0,qcd_040318,-27.0258,152.5642


## rainfall patterns

In [86]:
df_increments = pd.read_csv('../Inputs_FE_Assignment_3/ECsouth/ECsouth_Increments.csv')
df_increments.dropna(axis=0)
column_increments = [df_increments.columns.to_list()[index_2].replace(' ', '_') for index_2 in range(len(df_increments.columns.to_list()))]
column_increments = [column_increments[index_2].replace(':', '') for index_2 in range(len(column_increments))]
df_increments.rename(columns={df_increments.columns.to_list()[index_2]: column_increments[index_2] for index_2 in range(len(column_increments))}, inplace=True)
df_increments

,EventID,_Duration,_TimeStep,_Region,_AEP,_Increments,Unnamed_6,Unnamed_7,Unnamed_8,Unnamed_9,...,Unnamed_51,Unnamed_52,Unnamed_53,Unnamed_54,Unnamed_55,Unnamed_56,Unnamed_57,Unnamed_58,Unnamed_59,Unnamed_60
0,4380,10,5,East Coast (South),frequent,58.06,41.94,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4382,10,5,East Coast (South),frequent,52.13,47.87,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4384,10,5,East Coast (South),frequent,60.71,39.29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4385,10,5,East Coast (South),frequent,51.51,48.49,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4386,10,5,East Coast (South),frequent,54.55,45.45,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2856,10080,180,East Coast (South),rare,1.14,1.80,1.36,1.29,3.20,...,0.01,0.01,0.02,0.03,0.32,0.17,0.52,0.00,0.02,0.00
716,5064,10080,180,East Coast (South),rare,0.61,0.93,0.26,0.04,2.91,...,5.27,0.31,0.02,1.21,1.57,0.08,0.05,0.15,0.40,0.11
717,5069,10080,180,East Coast (South),rare,0.00,0.00,0.00,1.06,0.04,...,0.22,0.20,0.57,0.06,2.06,0.00,0.03,0.18,0.12,3.09
718,5071,10080,180,East Coast (South),rare,1.21,0.00,0.00,0.00,0.00,...,0.05,0.56,0.70,3.31,0.10,0.11,0.00,0.00,0.00,0.15


## precipitation depth

In [87]:
df_prec_depths = pd.read_csv('../Inputs_FE_Assignment_3/depths_33.8774_151.093/depths_-33.8774_151.093_all_design.csv', skiprows=9)
column_prec_depths = [df_prec_depths.columns.to_list()[index_3].replace(' ', '_') for index_3 in range(len(df_prec_depths.columns.to_list()))]
column_prec_depths = [column_prec_depths[index_3].replace('%', '_percentage') for index_3 in range(len(column_prec_depths))]
column_prec_depths = [column_prec_depths[index_3].replace('EY', '_EY') for index_3 in range(len(column_prec_depths))]
column_prec_depths = [column_prec_depths[index_3].replace('.', '_') for index_3 in range(len(column_prec_depths))]
df_prec_depths.rename(columns={df_prec_depths.columns.to_list()[index_3]: column_prec_depths[index_3] for index_3 in range(len(column_prec_depths))}, inplace=True)
df_prec_depths = pd.concat([df_prec_depths.iloc[:,:2], df_prec_depths.iloc[:,2:].add_prefix('freq_')], axis=1)
df_prec_depths

,Duration,Duration_in_min,freq_12_EY,freq_6_EY,freq_4_EY,freq_3_EY,freq_2_EY,freq_63_2_percentage,freq_50_percentage,freq_0_5_EY,freq_20_percentage,freq_0_2_EY,freq_10_percentage,freq_5_percentage,freq_2_percentage,freq_1_percentage,freq_1_in_200,freq_1_in_500,freq_1_in_1000,freq_1_in_2000
0,1 min,1.0,0.999,1.13,1.37,1.55,1.80,2.28,2.51,2.79,3.25,3.31,3.74,4.22,4.84,5.32,5.82,6.58,7.17,7.76
1,2 min,2.0,1.770,2.00,2.39,2.67,3.07,3.77,4.10,4.55,5.14,5.25,5.86,6.57,7.52,8.25,9.08,10.30,11.30,12.20
2,3 min,3.0,2.410,2.73,3.29,3.68,4.25,5.23,5.71,6.34,7.21,7.35,8.23,9.24,10.60,11.60,12.80,14.50,15.80,17.10
3,4 min,4.0,2.960,3.36,4.07,4.58,5.30,6.56,7.19,7.98,9.15,9.33,10.50,11.80,13.50,14.90,16.30,18.40,20.10,21.80
4,5 min,5.0,3.430,3.91,4.76,5.37,6.23,7.75,8.52,9.46,10.90,11.10,12.60,14.10,16.20,17.80,19.50,22.10,24.00,26.10
5,10 min,10.0,5.150,5.91,7.28,8.27,9.70,12.20,13.60,15.10,17.60,18.00,20.40,23.00,26.40,29.00,31.60,35.80,39.00,42.20
6,15 min,15.0,6.290,7.25,8.96,10.20,12.00,15.30,16.90,18.80,22.00,22.50,25.50,28.70,33.00,36.20,39.60,44.70,48.70,52.80
7,20 min,20.0,7.160,8.26,10.20,11.70,13.70,17.50,19.40,21.50,25.20,25.70,29.10,32.80,37.70,41.40,45.20,51.20,55.70,60.40
8,25 min,25.0,7.860,9.07,11.20,12.80,15.10,19.30,21.40,23.70,27.70,28.30,31.90,36.00,41.30,45.30,49.60,56.10,61.20,66.30
9,30 min,30.0,8.460,9.76,12.10,13.80,16.30,20.80,23.00,25.50,29.70,30.30,34.20,38.60,44.30,48.60,53.20,60.20,65.60,71.10


# data required

In [88]:
timestep = 5
timestep

5

In [89]:
starting_date = dt.datetime(year=2023, month=1, day=1, hour=0, minute=5, second=0)
starting_date

datetime.datetime(2023, 1, 1, 0, 5)

In [90]:
rain_gauge = 'RainSeries1'
rain_gauge

'RainSeries1'

In [91]:
orig_SWMM_file = '../Inputs_FE_Assignment_3/971007_SW5.inp'
orig_SWMM_file

'../Inputs_FE_Assignment_3/971007_SW5.inp'

In [92]:
output_directory = 'Output_SWMM_directory'
output_directory

'Output_SWMM_directory'

In [93]:
time_delta = dt.timedelta(minutes=timestep)
time_delta

datetime.timedelta(seconds=300)

In [94]:
max_duration = df_increments[df_increments._TimeStep == timestep]._Duration.max()
max_duration

120

In [95]:
min_duration = df_increments[df_increments._TimeStep == timestep]._Duration.min()
min_duration

10

In [96]:
AEP_frequency_req = pd.concat([AEP_frequency, pd.DataFrame({'frequency_descriptor':df_prec_depths.columns.to_list()[7:16]})], axis=1)
AEP_frequency_req

,frequency,EY,AEP_in_percentage,AEP_1_in_X_years,ARI,frequency_descriptor
0,frequent,1.00,63.21,1.58,1.00,freq_63_2_percentage
1,frequent,0.69,50.00,2.00,1.44,freq_50_percentage
2,frequent,0.50,39.35,2.54,2.00,freq_0_5_EY
3,frequent,0.22,20.00,5.00,4.48,freq_20_percentage
4,frequent,0.20,18.13,5.52,5.00,freq_0_2_EY
5,intermediate,0.11,10.00,10.00,9.49,freq_10_percentage
6,intermediate,0.05,5.00,20.00,19.50,freq_5_percentage
7,rare,0.02,2.00,50.00,49.50,freq_2_percentage
8,rare,0.01,1.00,100.00,99.50,freq_1_percentage


In [97]:
max_len_freq = AEP_frequency_req.frequency.apply(lambda arg: len(arg)).max()
max_len_freq

12

In [98]:
max_len_desc = AEP_frequency_req.frequency_descriptor.apply(lambda arg: len(arg)).max()
max_len_desc

20

## burst duration

In [99]:
df_stats = df_statistics[['Burst_Duration_min', 'Burst_Loading', 'AEP_Window']]
df_stats = df_stats[df_stats.Burst_Duration_min <= max_duration]
df_stats.reset_index(drop=True, inplace=True)
df_stats

,Burst_Duration_min,Burst_Loading,AEP_Window
0,10.0,1.0,frequent
1,10.0,1.0,frequent
2,10.0,1.0,frequent
3,10.0,2.0,frequent
4,10.0,2.0,frequent
...,...,...,...
265,120.0,2.0,rare
266,120.0,2.0,rare
267,120.0,2.0,rare
268,120.0,2.0,rare


In [100]:
grouping_by_duration_stats = df_stats.Burst_Duration_min.drop_duplicates().reset_index(drop=True)
grouping_by_duration_stats

0     10.0
1     15.0
2     20.0
3     25.0
4     30.0
5     45.0
6     60.0
7     90.0
8    120.0
Name: Burst_Duration_min, dtype: float64

In [101]:
grouping_by_AEP_stats = df_stats.AEP_Window.drop_duplicates().reset_index(drop=True)
grouping_by_AEP_stats

0        frequent
1    intermediate
2            rare
Name: AEP_Window, dtype: object

In [102]:
list_stats = []

for index_4 in grouping_by_duration_stats.index:
    a1 = df_stats[df_stats.Burst_Duration_min == grouping_by_duration_stats[index_4]].dropna(axis=1)
    for index_5 in grouping_by_AEP_stats.index:
        a2 = a1[a1.AEP_Window == grouping_by_AEP_stats[index_5]]
        a2.reset_index(drop=True, inplace=True)
        a3 = pd.DataFrame({'Duration': [grouping_by_duration_stats[index_4]], 'AEP': [grouping_by_AEP_stats[index_5]], 'burst_duration': [a2]})
        list_stats.append(a3)

list_stats = pd.concat(list_stats, ignore_index=True)
list_stats

,Duration,AEP,burst_duration
0,10.0,frequent,Burst_Duration_min Burst_Loading AEP_Windo...
1,10.0,intermediate,Burst_Duration_min Burst_Loading AEP_Wi...
2,10.0,rare,Burst_Duration_min Burst_Loading AEP_Windo...
3,15.0,frequent,Burst_Duration_min Burst_Loading AEP_Windo...
4,15.0,intermediate,Burst_Duration_min Burst_Loading AEP_Wi...
5,15.0,rare,Burst_Duration_min Burst_Loading AEP_Windo...
6,20.0,frequent,Burst_Duration_min Burst_Loading AEP_Windo...
7,20.0,intermediate,Burst_Duration_min Burst_Loading AEP_Wi...
8,20.0,rare,Burst_Duration_min Burst_Loading AEP_Windo...
9,25.0,frequent,Burst_Duration_min Burst_Loading AEP_Windo...


## temporal patterns

In [103]:
df_incre = df_increments.drop(columns=['EventID', '_Region'], axis=1)
df_incre.rename(columns={'_Duration':'Duration', '_TimeStep':'TimeStep', '_AEP':'AEP', '_Increments':'Increments'}, inplace=True)
df_incre = df_incre[(df_incre.TimeStep == timestep)]
df_incre.reset_index(drop=True, inplace=True)
df_incre.rename(columns={df_incre.columns[3:][index_6]: 'inc_{}'.format(index_6) for index_6 in range(len(df_incre.columns[3:]))}, inplace=True)
df_incre.drop(columns=df_incre.columns[df_incre.sum(axis=0) == 0], inplace=True)
df_incre

,Duration,TimeStep,AEP,inc_0,inc_1,inc_2,inc_3,inc_4,inc_5,inc_6,...,inc_14,inc_15,inc_16,inc_17,inc_18,inc_19,inc_20,inc_21,inc_22,inc_23
0,10,5,frequent,58.06,41.94,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,5,frequent,52.13,47.87,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,5,frequent,60.71,39.29,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,5,frequent,51.51,48.49,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,5,frequent,54.55,45.45,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,120,5,rare,2.07,2.58,2.57,1.54,1.03,3.60,3.08,...,8.74,6.68,2.06,0.77,0.77,3.08,5.14,4.63,1.03,1.80
266,120,5,rare,6.32,6.88,5.02,4.78,4.53,3.58,3.03,...,3.23,2.92,3.34,4.18,3.94,2.97,4.71,5.00,3.50,4.50
267,120,5,rare,7.53,12.55,8.58,2.62,2.62,1.19,0.76,...,6.38,6.22,8.84,7.93,7.93,3.86,1.13,0.59,0.45,0.28
268,120,5,rare,0.67,0.00,1.34,7.38,4.03,4.70,2.68,...,6.04,2.01,3.36,11.41,3.36,4.03,2.68,7.38,5.37,6.71


In [104]:
grouping_by_duration_incre = df_incre.Duration.drop_duplicates().reset_index(drop=True)
grouping_by_duration_incre

0     10
1     15
2     20
3     25
4     30
5     45
6     60
7     90
8    120
Name: Duration, dtype: int64

In [105]:
grouping_by_AEP_incre = df_incre.AEP.drop_duplicates().reset_index(drop=True)
grouping_by_AEP_incre

0        frequent
1    intermediate
2            rare
Name: AEP, dtype: object

In [106]:
list_incre = []

for index_7 in grouping_by_duration_incre.index:
    b1 = df_incre[df_incre.Duration == grouping_by_duration_incre[index_7]].dropna(axis=1)
    for index_8 in grouping_by_AEP_incre.index:
        b2 = b1[b1.AEP == grouping_by_AEP_incre[index_8]]
        b2.reset_index(drop=True, inplace=True)
        b3 = pd.DataFrame({'Duration': [grouping_by_duration_incre[index_7]], 'AEP': [grouping_by_AEP_incre[index_8]], 'increments': [b2]})
        list_incre.append(b3)

list_incre = pd.concat(list_incre, ignore_index=True)
list_incre

,Duration,AEP,increments
0,10,frequent,Duration TimeStep AEP inc_0 inc_1 ...
1,10,intermediate,Duration TimeStep AEP inc_0 in...
2,10,rare,Duration TimeStep AEP inc_0 inc_1 0 ...
3,15,frequent,Duration TimeStep AEP inc_0 inc_1 ...
4,15,intermediate,Duration TimeStep AEP inc_0 in...
5,15,rare,Duration TimeStep AEP inc_0 inc_1 inc...
6,20,frequent,Duration TimeStep AEP inc_0 inc_1 ...
7,20,intermediate,Duration TimeStep AEP inc_0 in...
8,20,rare,Duration TimeStep AEP inc_0 inc_1 inc...
9,25,frequent,Duration TimeStep AEP inc_0 inc_1 ...


## precipitation depth

In [107]:
df_depth = df_prec_depths[['Duration_in_min'] + AEP_frequency_req.frequency_descriptor.to_list()]
df_depth = df_depth[(df_depth.Duration_in_min >= min_duration) & (df_depth.Duration_in_min <= max_duration)]
df_depth.reset_index(drop=True, inplace=True)
df_depth

,Duration_in_min,freq_63_2_percentage,freq_50_percentage,freq_0_5_EY,freq_20_percentage,freq_0_2_EY,freq_10_percentage,freq_5_percentage,freq_2_percentage,freq_1_percentage
0,10.0,12.2,13.6,15.1,17.6,18.0,20.4,23.0,26.4,29.0
1,15.0,15.3,16.9,18.8,22.0,22.5,25.5,28.7,33.0,36.2
2,20.0,17.5,19.4,21.5,25.2,25.7,29.1,32.8,37.7,41.4
3,25.0,19.3,21.4,23.7,27.7,28.3,31.9,36.0,41.3,45.3
4,30.0,20.8,23.0,25.5,29.7,30.3,34.2,38.6,44.3,48.6
5,45.0,24.3,26.7,29.7,34.3,35.0,39.4,44.4,50.9,55.9
6,60.0,27.0,29.5,32.8,37.7,38.5,43.3,48.7,55.9,61.5
7,90.0,31.0,33.9,37.6,43.0,43.9,49.3,55.6,64.0,70.5
8,120.0,34.3,37.5,41.6,47.5,48.4,54.4,61.4,70.8,78.2


# temporal patterns

In [108]:
list_temp_patterns = []

for index_9 in list_stats.index:
    c1 = list_stats.Duration[index_9]
    c2 = list_stats.AEP[index_9]
    c3 = pd.concat([list_stats.burst_duration[index_9].iloc[:,:2], list_incre.increments[index_9].iloc[:,1:]], axis=1)
    c4 = pd.DataFrame({'Duration':[c1], 'AEP':[c2], 'temp_patterns':[c3]})
    list_temp_patterns.append(c4)

list_temp_patterns = pd.concat(list_temp_patterns, ignore_index=True)
list_temp_patterns.sort_values(by=['AEP', 'Duration'], inplace=True, ignore_index=True)
list_temp_patterns

,Duration,AEP,temp_patterns
0,10.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
1,15.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
2,20.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
3,25.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
4,30.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
5,45.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
6,60.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
7,90.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
8,120.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
9,10.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...


# frequency

## frequent

In [109]:
freq_temp_patterns = pd.concat([df_depth[['Duration_in_min'] + AEP_frequency_req[AEP_frequency_req.frequency == 'frequent'].frequency_descriptor.to_list()], 
                                list_temp_patterns[list_temp_patterns.AEP == 'frequent'].reset_index(drop=True)], axis=1)
freq_temp_patterns

,Duration_in_min,freq_63_2_percentage,freq_50_percentage,freq_0_5_EY,freq_20_percentage,freq_0_2_EY,Duration,AEP,temp_patterns
0,10.0,12.2,13.6,15.1,17.6,18.0,10.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
1,15.0,15.3,16.9,18.8,22.0,22.5,15.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
2,20.0,17.5,19.4,21.5,25.2,25.7,20.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
3,25.0,19.3,21.4,23.7,27.7,28.3,25.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
4,30.0,20.8,23.0,25.5,29.7,30.3,30.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
5,45.0,24.3,26.7,29.7,34.3,35.0,45.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
6,60.0,27.0,29.5,32.8,37.7,38.5,60.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
7,90.0,31.0,33.9,37.6,43.0,43.9,90.0,frequent,Burst_Duration_min Burst_Loading TimeStep...
8,120.0,34.3,37.5,41.6,47.5,48.4,120.0,frequent,Burst_Duration_min Burst_Loading TimeStep...


In [110]:
freq_hyetograph_depths = []

for index_10 in freq_temp_patterns.index:
    list_temp = []
    for index_11 in AEP_frequency_req[AEP_frequency_req.frequency == 'frequent'].frequency_descriptor.index:
        d1 = freq_temp_patterns[AEP_frequency_req[AEP_frequency_req.frequency == 'frequent'].frequency_descriptor[index_11]][index_10]
        d2 = freq_temp_patterns.temp_patterns[index_10].iloc[:,4:]
        d3 = d1 * d2 / 100
        d4 = pd.DataFrame({AEP_frequency_req[AEP_frequency_req.frequency == 'frequent'].frequency_descriptor[index_11]:[d3]})
        list_temp.append(d4)
    list_temp = pd.concat(list_temp, axis=1)
    freq_hyetograph_depths.append(list_temp)

freq_hyetograph_depths = pd.concat(freq_hyetograph_depths, ignore_index=True)
freq_hyetograph_depths

,freq_63_2_percentage,freq_50_percentage,freq_0_5_EY,freq_20_percentage,freq_0_2_EY
0,inc_0 inc_1 0 7.08332 5.11668 1 6.3...,inc_0 inc_1 0 7.89616 5.70384 1 7.0...,inc_0 inc_1 0 8.76706 6.33294 1 7...,inc_0 inc_1 0 10.21856 7.38144 1 ...,inc_0 inc_1 0 10.4508 7.5492 1 9....
1,inc_0 inc_1 inc_2 0 7.20783 4.299...,inc_0 inc_1 inc_2 0 7.96159 4.748...,inc_0 inc_1 inc_2 0 8.85668 5.282...,inc_0 inc_1 inc_2 0 10.3642 6.1820...,inc_0 inc_1 inc_2 0 10.59975 6....
2,inc_0 inc_1 inc_2 inc_3 0 6.168...,inc_0 inc_1 inc_2 inc_3 0 6.838...,inc_0 inc_1 inc_2 inc_3 0 7.578...,inc_0 inc_1 inc_2 inc_3 0 8...,inc_0 inc_1 inc_2 inc_3 0 9...
3,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_...,inc_0 inc_1 inc_2 inc_3 in...,inc_0 inc_1 inc_2 inc_3 in...
4,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4 in...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc...,inc_0 inc_1 inc_2 inc_3 inc...
5,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4 inc...
6,inc_0 inc_1 inc_2 inc_3 inc_4 in...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...
7,inc_0 inc_1 inc_2 inc_3 inc_4 in...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4 in...,inc_0 inc_1 inc_2 inc_3 inc_4...
8,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...


In [111]:
freq_hyetograph_timestamp = []

for index_12 in freq_temp_patterns.index:
    list_temp = []
    e1 = freq_temp_patterns.temp_patterns[index_12].columns[4:]
    for index_13 in range(int(freq_temp_patterns.Duration[index_12] / timestep)):
        e2 = starting_date + index_13 * time_delta
        e3 = pd.DataFrame({e1[index_13]: [e2]})
        list_temp.append(e3)
    list_temp = pd.concat(list_temp, axis=1)
    e4 = pd.DataFrame({'increments': [list_temp]})
    freq_hyetograph_timestamp.append(e4)

freq_hyetograph_timestamp = pd.concat(freq_hyetograph_timestamp, ignore_index=True)
freq_hyetograph_timestamp

,increments
0,inc_0 inc_1 0 20...
1,inc_0 inc_1 ...
2,inc_0 inc_1 ...
3,inc_0 inc_1 ...
4,inc_0 inc_1 ...
5,inc_0 inc_1 ...
6,inc_0 inc_1 ...
7,inc_0 inc_1 ...
8,inc_0 inc_1 ...


In [112]:
freq_rainseries = pd.concat([freq_temp_patterns.Duration_in_min, freq_temp_patterns.AEP, freq_hyetograph_timestamp, freq_hyetograph_depths], axis=1)
freq_rainseries

,Duration_in_min,AEP,increments,freq_63_2_percentage,freq_50_percentage,freq_0_5_EY,freq_20_percentage,freq_0_2_EY
0,10.0,frequent,inc_0 inc_1 0 20...,inc_0 inc_1 0 7.08332 5.11668 1 6.3...,inc_0 inc_1 0 7.89616 5.70384 1 7.0...,inc_0 inc_1 0 8.76706 6.33294 1 7...,inc_0 inc_1 0 10.21856 7.38144 1 ...,inc_0 inc_1 0 10.4508 7.5492 1 9....
1,15.0,frequent,inc_0 inc_1 ...,inc_0 inc_1 inc_2 0 7.20783 4.299...,inc_0 inc_1 inc_2 0 7.96159 4.748...,inc_0 inc_1 inc_2 0 8.85668 5.282...,inc_0 inc_1 inc_2 0 10.3642 6.1820...,inc_0 inc_1 inc_2 0 10.59975 6....
2,20.0,frequent,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 0 6.168...,inc_0 inc_1 inc_2 inc_3 0 6.838...,inc_0 inc_1 inc_2 inc_3 0 7.578...,inc_0 inc_1 inc_2 inc_3 0 8...,inc_0 inc_1 inc_2 inc_3 0 9...
3,25.0,frequent,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_...,inc_0 inc_1 inc_2 inc_3 in...,inc_0 inc_1 inc_2 inc_3 in...
4,30.0,frequent,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4 in...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc...,inc_0 inc_1 inc_2 inc_3 inc...
5,45.0,frequent,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4 inc...
6,60.0,frequent,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4 in...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...
7,90.0,frequent,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4 in...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4 in...,inc_0 inc_1 inc_2 inc_3 inc_4...
8,120.0,frequent,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...


## intermediate

In [113]:
inte_temp_patterns = pd.concat([df_depth[['Duration_in_min'] + AEP_frequency_req[AEP_frequency_req.frequency == 'intermediate'].frequency_descriptor.to_list()], 
                                list_temp_patterns[list_temp_patterns.AEP == 'intermediate'].reset_index(drop=True)], axis=1)
inte_temp_patterns

,Duration_in_min,freq_10_percentage,freq_5_percentage,Duration,AEP,temp_patterns
0,10.0,20.4,23.0,10.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...
1,15.0,25.5,28.7,15.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...
2,20.0,29.1,32.8,20.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...
3,25.0,31.9,36.0,25.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...
4,30.0,34.2,38.6,30.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...
5,45.0,39.4,44.4,45.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...
6,60.0,43.3,48.7,60.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...
7,90.0,49.3,55.6,90.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...
8,120.0,54.4,61.4,120.0,intermediate,Burst_Duration_min Burst_Loading TimeStep...


In [114]:
inte_hyetograph_depths = []

for index_10 in inte_temp_patterns.index:
    list_temp = []
    for index_11 in AEP_frequency_req[AEP_frequency_req.frequency == 'intermediate'].frequency_descriptor.index:
        d1 = inte_temp_patterns[AEP_frequency_req[AEP_frequency_req.frequency == 'intermediate'].frequency_descriptor[index_11]][index_10]
        d2 = inte_temp_patterns.temp_patterns[index_10].iloc[:,4:]
        d3 = d1 * d2 / 100
        d4 = pd.DataFrame({AEP_frequency_req[AEP_frequency_req.frequency == 'intermediate'].frequency_descriptor[index_11]:[d3]})
        list_temp.append(d4)
    list_temp = pd.concat(list_temp, axis=1)
    inte_hyetograph_depths.append(list_temp)

inte_hyetograph_depths = pd.concat(inte_hyetograph_depths, ignore_index=True)
inte_hyetograph_depths

,freq_10_percentage,freq_5_percentage
0,inc_0 inc_1 0 11.26488 9.13512 1 ...,inc_0 inc_1 0 12.7006 10.2994 1 14....
1,inc_0 inc_1 inc_2 0 11.47500 ...,inc_0 inc_1 inc_2 0 12.91500 ...
2,inc_0 inc_1 inc_2 inc_3 0 ...,inc_0 inc_1 inc_2 inc_3 0 1...
3,inc_0 inc_1 inc_2 inc_3 in...,inc_0 inc_1 inc_2 inc_3 inc_4 ...
4,inc_0 inc_1 inc_2 inc_3 inc_...,inc_0 inc_1 inc_2 inc_3 ...
5,inc_0 inc_1 inc_2 inc_3 inc...,inc_0 inc_1 inc_2 inc_3 inc...
6,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...
7,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_...
8,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...


In [115]:
inte_hyetograph_timestamp = []

for index_12 in inte_temp_patterns.index:
    list_temp = []
    e1 = inte_temp_patterns.temp_patterns[index_12].columns[4:]
    for index_13 in range(int(inte_temp_patterns.Duration[index_12] / timestep)):
        e2 = starting_date + index_13 * time_delta
        e3 = pd.DataFrame({e1[index_13]: [e2]})
        list_temp.append(e3)
    list_temp = pd.concat(list_temp, axis=1)
    e4 = pd.DataFrame({'increments': [list_temp]})
    inte_hyetograph_timestamp.append(e4)

inte_hyetograph_timestamp = pd.concat(inte_hyetograph_timestamp, ignore_index=True)
inte_hyetograph_timestamp

,increments
0,inc_0 inc_1 0 20...
1,inc_0 inc_1 ...
2,inc_0 inc_1 ...
3,inc_0 inc_1 ...
4,inc_0 inc_1 ...
5,inc_0 inc_1 ...
6,inc_0 inc_1 ...
7,inc_0 inc_1 ...
8,inc_0 inc_1 ...


In [116]:
inte_rainseries = pd.concat([inte_temp_patterns.Duration_in_min, inte_temp_patterns.AEP, inte_hyetograph_timestamp, inte_hyetograph_depths], axis=1)
inte_rainseries

,Duration_in_min,AEP,increments,freq_10_percentage,freq_5_percentage
0,10.0,intermediate,inc_0 inc_1 0 20...,inc_0 inc_1 0 11.26488 9.13512 1 ...,inc_0 inc_1 0 12.7006 10.2994 1 14....
1,15.0,intermediate,inc_0 inc_1 ...,inc_0 inc_1 inc_2 0 11.47500 ...,inc_0 inc_1 inc_2 0 12.91500 ...
2,20.0,intermediate,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 0 ...,inc_0 inc_1 inc_2 inc_3 0 1...
3,25.0,intermediate,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 in...,inc_0 inc_1 inc_2 inc_3 inc_4 ...
4,30.0,intermediate,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_...,inc_0 inc_1 inc_2 inc_3 ...
5,45.0,intermediate,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc...,inc_0 inc_1 inc_2 inc_3 inc...
6,60.0,intermediate,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...
7,90.0,intermediate,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_...
8,120.0,intermediate,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4...,inc_0 inc_1 inc_2 inc_3 inc_4...


## rare

In [117]:
rare_temp_patterns = pd.concat([df_depth[['Duration_in_min'] + AEP_frequency_req[AEP_frequency_req.frequency == 'rare'].frequency_descriptor.to_list()], 
                                list_temp_patterns[list_temp_patterns.AEP == 'rare'].reset_index(drop=True)], axis=1)
rare_temp_patterns

,Duration_in_min,freq_2_percentage,freq_1_percentage,Duration,AEP,temp_patterns
0,10.0,26.4,29.0,10.0,rare,Burst_Duration_min Burst_Loading TimeStep...
1,15.0,33.0,36.2,15.0,rare,Burst_Duration_min Burst_Loading TimeStep...
2,20.0,37.7,41.4,20.0,rare,Burst_Duration_min Burst_Loading TimeStep...
3,25.0,41.3,45.3,25.0,rare,Burst_Duration_min Burst_Loading TimeStep...
4,30.0,44.3,48.6,30.0,rare,Burst_Duration_min Burst_Loading TimeStep...
5,45.0,50.9,55.9,45.0,rare,Burst_Duration_min Burst_Loading TimeStep...
6,60.0,55.9,61.5,60.0,rare,Burst_Duration_min Burst_Loading TimeStep...
7,90.0,64.0,70.5,90.0,rare,Burst_Duration_min Burst_Loading TimeStep...
8,120.0,70.8,78.2,120.0,rare,Burst_Duration_min Burst_Loading TimeStep...


In [118]:
rare_hyetograph_depths = []

for index_10 in rare_temp_patterns.index:
    list_temp = []
    for index_11 in AEP_frequency_req[AEP_frequency_req.frequency == 'rare'].frequency_descriptor.index:
        d1 = rare_temp_patterns[AEP_frequency_req[AEP_frequency_req.frequency == 'rare'].frequency_descriptor[index_11]][index_10]
        d2 = rare_temp_patterns.temp_patterns[index_10].iloc[:,4:]
        d3 = d1 * d2 / 100
        d4 = pd.DataFrame({AEP_frequency_req[AEP_frequency_req.frequency == 'rare'].frequency_descriptor[index_11]:[d3]})
        list_temp.append(d4)
    list_temp = pd.concat(list_temp, axis=1)
    rare_hyetograph_depths.append(list_temp)

rare_hyetograph_depths = pd.concat(rare_hyetograph_depths, ignore_index=True)
rare_hyetograph_depths

,freq_2_percentage,freq_1_percentage
0,inc_0 inc_1 0 15.98784 10.41216 1 ...,inc_0 inc_1 0 17.5624 11.4376 1 17....
1,inc_0 inc_1 inc_2 0 8.9397 11.89...,inc_0 inc_1 inc_2 0 9.80658 1...
2,inc_0 inc_1 inc_2 inc_3 0 ...,inc_0 inc_1 inc_2 inc_3 0 ...
3,inc_0 inc_1 inc_2 inc_3 i...,inc_0 inc_1 inc_2 inc_3 ...
4,inc_0 inc_1 inc_2 inc_3 ...,inc_0 inc_1 inc_2 inc_3 ...
5,inc_0 inc_1 inc_2 inc_3 inc...,inc_0 inc_1 inc_2 inc_3 in...
6,inc_0 inc_1 inc_2 inc_3 inc...,inc_0 inc_1 inc_2 inc_3 inc...
7,inc_0 inc_1 inc_2 inc_3 inc_4 ...,inc_0 inc_1 inc_2 inc_3 inc_...
8,inc_0 inc_1 inc_2 inc_3 inc_...,inc_0 inc_1 inc_2 inc_3 inc_...


In [119]:
rare_hyetograph_timestamp = []

for index_12 in rare_temp_patterns.index:
    list_temp = []
    e1 = rare_temp_patterns.temp_patterns[index_12].columns[4:]
    for index_13 in range(int(rare_temp_patterns.Duration[index_12] / timestep)):
        e2 = starting_date + index_13 * time_delta
        e3 = pd.DataFrame({e1[index_13]: [e2]})
        list_temp.append(e3)
    list_temp = pd.concat(list_temp, axis=1)
    e4 = pd.DataFrame({'increments': [list_temp]})
    rare_hyetograph_timestamp.append(e4)

rare_hyetograph_timestamp = pd.concat(rare_hyetograph_timestamp, ignore_index=True)
rare_hyetograph_timestamp

,increments
0,inc_0 inc_1 0 20...
1,inc_0 inc_1 ...
2,inc_0 inc_1 ...
3,inc_0 inc_1 ...
4,inc_0 inc_1 ...
5,inc_0 inc_1 ...
6,inc_0 inc_1 ...
7,inc_0 inc_1 ...
8,inc_0 inc_1 ...


In [120]:
rare_rainseries = pd.concat([rare_temp_patterns.Duration_in_min, rare_temp_patterns.AEP, rare_hyetograph_timestamp, rare_hyetograph_depths], axis=1)
rare_rainseries

,Duration_in_min,AEP,increments,freq_2_percentage,freq_1_percentage
0,10.0,rare,inc_0 inc_1 0 20...,inc_0 inc_1 0 15.98784 10.41216 1 ...,inc_0 inc_1 0 17.5624 11.4376 1 17....
1,15.0,rare,inc_0 inc_1 ...,inc_0 inc_1 inc_2 0 8.9397 11.89...,inc_0 inc_1 inc_2 0 9.80658 1...
2,20.0,rare,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 0 ...,inc_0 inc_1 inc_2 inc_3 0 ...
3,25.0,rare,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 i...,inc_0 inc_1 inc_2 inc_3 ...
4,30.0,rare,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 ...,inc_0 inc_1 inc_2 inc_3 ...
5,45.0,rare,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc...,inc_0 inc_1 inc_2 inc_3 in...
6,60.0,rare,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc...,inc_0 inc_1 inc_2 inc_3 inc...
7,90.0,rare,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_4 ...,inc_0 inc_1 inc_2 inc_3 inc_...
8,120.0,rare,inc_0 inc_1 ...,inc_0 inc_1 inc_2 inc_3 inc_...,inc_0 inc_1 inc_2 inc_3 inc_...


# list frequencies

## frequent

In [121]:
freq_list = []

# for each column: frequency AEP column
for index_14 in freq_rainseries.columns[3:]:
    f0 = pd.DataFrame(freq_rainseries[index_14])

    # for each row: duration (9 categories), AEP (3 categories - 1 for this)
    for index_15 in f0.index:
        f1 = freq_rainseries.AEP[index_15]
        f2 = int(freq_rainseries.Duration_in_min[index_15])
        f3 = freq_rainseries.increments[index_15]
        f4 = f0[index_14][index_15]

        # for each row: temporal pattern (10 temporal patterns)
        for index_16 in f4.index:
            f5 = pd.DataFrame(f4.loc[index_16]).T
            temp_list = []

            # for each column: depending on duration - 2, 3, 4, 5, 6, 9, 12, 18, 24 timestamps
            for index_17 in f5.columns:
                f6 = dt.datetime.strftime(f3[index_17][0], '%m/%d/%Y %H:%M')
                f7 = f4[index_17][index_16]
                f8 = pd.DataFrame({'hydrograph':['{}      {}      {}'.format(rain_gauge, f6, f7)]})
                temp_list.append(f8)

            temp_list = pd.concat(temp_list, ignore_index=True)
            f9 = '{}__{}__{:03}__{:02}'.format(f1 if len(f1) == max_len_freq else f1 + (max_len_freq - len(f1)) * '_', 
                                               index_14 if len(index_14) == max_len_desc else index_14 + (max_len_desc - len(index_14)) * '_', 
                                               f2, index_16 + 1)
            f10 = pd.DataFrame({'frequency_AEP':[f1], 'AEP':[index_14], 'Duration':[f2], 'pattern':[index_16 + 1], 'name':[f9], 'timeserie': [temp_list]})
            freq_list.append(f10)

freq_list = pd.concat(freq_list, ignore_index=True)
freq_list

,frequency_AEP,AEP,Duration,pattern,name,timeserie
0,frequent,freq_63_2_percentage,10,1,frequent______freq_63_2_percentage__010__01,hydrogr...
1,frequent,freq_63_2_percentage,10,2,frequent______freq_63_2_percentage__010__02,hydr...
2,frequent,freq_63_2_percentage,10,3,frequent______freq_63_2_percentage__010__03,hydr...
3,frequent,freq_63_2_percentage,10,4,frequent______freq_63_2_percentage__010__04,hydr...
4,frequent,freq_63_2_percentage,10,5,frequent______freq_63_2_percentage__010__05,hydr...
...,...,...,...,...,...,...
445,frequent,freq_0_2_EY,120,6,frequent______freq_0_2_EY___________120__06,hyd...
446,frequent,freq_0_2_EY,120,7,frequent______freq_0_2_EY___________120__07,hyd...
447,frequent,freq_0_2_EY,120,8,frequent______freq_0_2_EY___________120__08,hyd...
448,frequent,freq_0_2_EY,120,9,frequent______freq_0_2_EY___________120__09,hyd...


## intermediate

In [122]:
inte_list = []

# for each column: frequency AEP column
for index_14 in inte_rainseries.columns[3:]:
    f0 = pd.DataFrame(inte_rainseries[index_14])

    # for each row: duration (9 categories), AEP (3 categories - 1 for this)
    for index_15 in f0.index:
        f1 = inte_rainseries.AEP[index_15]
        f2 = int(inte_rainseries.Duration_in_min[index_15])
        f3 = inte_rainseries.increments[index_15]
        f4 = f0[index_14][index_15]

        # for each row: temporal pattern (10 temporal patterns)
        for index_16 in f4.index:
            f5 = pd.DataFrame(f4.loc[index_16]).T
            temp_list = []

            # for each column: depending on duration - 2, 3, 4, 5, 6, 9, 12, 18, 24 timestamps
            for index_17 in f5.columns:
                f6 = dt.datetime.strftime(f3[index_17][0], '%m/%d/%Y %H:%M')
                f7 = f4[index_17][index_16]
                f8 = pd.DataFrame({'hydrograph':['{}      {}      {}'.format(rain_gauge, f6, f7)]})
                temp_list.append(f8)

            temp_list = pd.concat(temp_list, ignore_index=True)
            f9 = '{}__{}__{:03}__{:02}'.format(f1 if len(f1) == max_len_freq else f1 + (max_len_freq - len(f1)) * '_', 
                                               index_14 if len(index_14) == max_len_desc else index_14 + (max_len_desc - len(index_14)) * '_', 
                                               f2, index_16 + 1)
            f10 = pd.DataFrame({'frequency_AEP':[f1], 'AEP':[index_14], 'Duration':[f2], 'pattern':[index_16 + 1], 'name':[f9], 'timeserie': [temp_list]})
            inte_list.append(f10)

inte_list = pd.concat(inte_list, ignore_index=True)
inte_list

,frequency_AEP,AEP,Duration,pattern,name,timeserie
0,intermediate,freq_10_percentage,10,1,intermediate__freq_10_percentage____010__01,hydr...
1,intermediate,freq_10_percentage,10,2,intermediate__freq_10_percentage____010__02,hydrog...
2,intermediate,freq_10_percentage,10,3,intermediate__freq_10_percentage____010__03,hydr...
3,intermediate,freq_10_percentage,10,4,intermediate__freq_10_percentage____010__04,hydr...
4,intermediate,freq_10_percentage,10,5,intermediate__freq_10_percentage____010__05,hydr...
...,...,...,...,...,...,...
175,intermediate,freq_5_percentage,120,6,intermediate__freq_5_percentage_____120__06,hyd...
176,intermediate,freq_5_percentage,120,7,intermediate__freq_5_percentage_____120__07,hyd...
177,intermediate,freq_5_percentage,120,8,intermediate__freq_5_percentage_____120__08,hyd...
178,intermediate,freq_5_percentage,120,9,intermediate__freq_5_percentage_____120__09,hyd...


## rare

In [123]:
rare_list = []

# for each column: frequency AEP column
for index_14 in rare_rainseries.columns[3:]:
    f0 = pd.DataFrame(rare_rainseries[index_14])

    # for each row: duration (9 categories), AEP (3 categories - 1 for this)
    for index_15 in f0.index:
        f1 = rare_rainseries.AEP[index_15]
        f2 = int(rare_rainseries.Duration_in_min[index_15])
        f3 = rare_rainseries.increments[index_15]
        f4 = f0[index_14][index_15]

        # for each row: temporal pattern (10 temporal patterns)
        for index_16 in f4.index:
            f5 = pd.DataFrame(f4.loc[index_16]).T
            temp_list = []

            # for each column: depending on duration - 2, 3, 4, 5, 6, 9, 12, 18, 24 timestamps
            for index_17 in f5.columns:
                f6 = dt.datetime.strftime(f3[index_17][0], '%m/%d/%Y %H:%M')
                f7 = f4[index_17][index_16]
                f8 = pd.DataFrame({'hydrograph':['{}      {}      {}'.format(rain_gauge, f6, f7)]})
                temp_list.append(f8)

            temp_list = pd.concat(temp_list, ignore_index=True)
            f9 = '{}__{}__{:03}__{:02}'.format(f1 if len(f1) == max_len_freq else f1 + (max_len_freq - len(f1)) * '_', 
                                               index_14 if len(index_14) == max_len_desc else index_14 + (max_len_desc - len(index_14)) * '_', 
                                               f2, index_16 + 1)
            f10 = pd.DataFrame({'frequency_AEP':[f1], 'AEP':[index_14], 'Duration':[f2], 'pattern':[index_16 + 1], 'name':[f9], 'timeserie': [temp_list]})
            rare_list.append(f10)

rare_list = pd.concat(rare_list, ignore_index=True)
rare_list

,frequency_AEP,AEP,Duration,pattern,name,timeserie
0,rare,freq_2_percentage,10,1,rare__________freq_2_percentage_____010__01,hydr...
1,rare,freq_2_percentage,10,2,rare__________freq_2_percentage_____010__02,hydrogr...
2,rare,freq_2_percentage,10,3,rare__________freq_2_percentage_____010__03,hydr...
3,rare,freq_2_percentage,10,4,rare__________freq_2_percentage_____010__04,hydrog...
4,rare,freq_2_percentage,10,5,rare__________freq_2_percentage_____010__05,hydrog...
...,...,...,...,...,...,...
175,rare,freq_1_percentage,120,6,rare__________freq_1_percentage_____120__06,hyd...
176,rare,freq_1_percentage,120,7,rare__________freq_1_percentage_____120__07,hyd...
177,rare,freq_1_percentage,120,8,rare__________freq_1_percentage_____120__08,hyd...
178,rare,freq_1_percentage,120,9,rare__________freq_1_percentage_____120__09,hyd...


# generate outputs

## output directory

In [124]:
if not os.path.exists('../' + output_directory):
    os.makedirs('../' + output_directory)

## frequent

In [729]:
for index_18 in freq_list.index:
    g1 = '../' + output_directory + '/' + freq_list.name[index_18] + '.inp'
    shutil.copyfile(orig_SWMM_file, g1)
    g2 = freq_list.timeserie[index_18]
    for index_19 in g2.index:
        g3 = g2.hydrograph[index_19]
        with open(g1, 'r+') as SWMM_file:
            g4 = SWMM_file.readlines()
            g4.insert(704 + index_19, g3 + '\n')
            SWMM_file.seek(0)
            SWMM_file.writelines(g4)

## intermediate

In [730]:
for index_18 in inte_list.index:
    g1 = '../' + output_directory + '/' + inte_list.name[index_18] + '.inp'
    shutil.copyfile(orig_SWMM_file, g1)
    g2 = inte_list.timeserie[index_18]
    for index_19 in g2.index:
        g3 = g2.hydrograph[index_19]
        with open(g1, 'r+') as SWMM_file:
            g4 = SWMM_file.readlines()
            g4.insert(704 + index_19, g3 + '\n')
            SWMM_file.seek(0)
            SWMM_file.writelines(g4)

## rare

In [731]:
for index_18 in rare_list.index:
    g1 = '../' + output_directory + '/' + rare_list.name[index_18] + '.inp'
    shutil.copyfile(orig_SWMM_file, g1)
    g2 = rare_list.timeserie[index_18]
    for index_19 in g2.index:
        g3 = g2.hydrograph[index_19]
        with open(g1, 'r+') as SWMM_file:
            g4 = SWMM_file.readlines()
            g4.insert(704 + index_19, g3 + '\n')
            SWMM_file.seek(0)
            SWMM_file.writelines(g4)